# Run network model

In [ ]:
#%load_ext autoreload
#%autoreload 2

#from keras.layers import merge
from src.models.data import *
from src.models.model import *
from src.data.utils import get_tile_prefix
from src.models.metrics_img import auc_roc
#import rasterio.plot as rioplot
import matplotlib
import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

from pathlib import Path
import os, shutil
import sys
%matplotlib inline

## User settings

In [ ]:
# paths to append
sys.path.append("/home/ubuntu/roaddetection/")
sys.path.append("/media/hh/hd_internal/hh/DSR_Berlin_2018/roaddetection/")

# ------------- image characteristics and augmentation -----------------------------
# size of tiles
target_size = (512,512)
# input arguments to Keras' ImageDataGenerator
data_gen_args = dict(
                    data_format="channels_last",
                    horizontal_flip=True, 
                    vertical_flip=True
 )
# if True, image tiles will be split up into training/validation/test sets
# only required after a fresh 'make data'
do_data_split = False
# directory into which to place *training* images from ImageDataGenerator for inspection;
# default should be None because this slows things down
imgdatagen_dir = None
#imgdatagen_dir = '../../data/imgdatagenerator'

#--------------- network weights ----------------------------------------------------
# path to & filename of pre-trained model to use - set to None if you want to start from scratch
pretrained_model_fn = "../../models/unet_membrane_analytic_27_08_14_55.hdf5"
#pretrained_model_fn = None

# path to & filename of model to save
trained_model_fn = '../../models/unet_membrane_analytic_.hdf5'

#--------------- training details / hyperparameters -----------------------------------
# batch size
batch_size = 2
# steps per epoch, should correspond to [number of training images] / batch size
steps_per_epoch = 100
# number of epochs
epochs = 15
# number of steps on validation set
validation_steps = 20
# self-explanatory variables:
optimizer = Adam(lr=1e-4)
loss = 'binary_crossentropy'
loss_weights = None
metrics = ['accuracy', auc_roc]

### Define and make (if necessary) train/validation/test directories

In [ ]:
original_dataset_dir = "../../data/train"
raw_images_path = "../../data/raw/images"
dirs = []

base_dir = "../../data"

train_dir = os.path.join(base_dir, "train")
dirs.append(train_dir)
validation_dir = os.path.join(base_dir, "validate")
dirs.append(validation_dir)
test_dir = os.path.join(base_dir, "test")
dirs.append(test_dir)

In [ ]:
for directory in dirs:
    if not os.path.exists(directory):
        os.mkdir(directory)

### Split data up into train/validation/test images

In [ ]:
if do_data_split:
    def should_make_tiles_from(r_analytic_name):
        is_analytic_tif = r_analytic_name.endswith(
            ('AnalyticMS.tif', 'AnalyticMS_SR.tif', 'AnalyticMS.tiff', 'AnalyticMS_SR.tiff')
        )
        return is_analytic_tif 

    file_prefixes = [ get_tile_prefix(r_analytic.name) 
                      for r_analytic in Path(raw_images_path).iterdir()  
                        if  should_make_tiles_from(r_analytic.name)
                    ]
    print(file_prefixes)

    # move files to validation dir
    validation_fnames = ["{0:s}_{1:04d}.tif".format(prefix, i) for i in range(1,10) for prefix in file_prefixes]
    for fname in validation_fnames:
        for file_type in ["sat", "map", "sat_rgb"]:
            src = os.path.join(train_dir, file_type, fname)
            dest = os.path.join(validation_dir, file_type, fname)
            shutil.move(src, dest)

    # move files to test dir
    test_fnames = ["{0:s}_{1:04d}.tif".format(prefix, i) for i in range(40,60) for prefix in file_prefixes]
    for fname in test_fnames:
        for file_type in ["sat", "map", "sat_rgb"]:
            src = os.path.join(train_dir, file_type, fname)
            dest = os.path.join(test_dir, file_type, fname)
            shutil.move(src, dest)

    # print overview
    for directory in dirs:
        for file_type in ["sat", "map", "sat_rgb"]:
            target = os.path.join(directory, file_type)
            print(target, ":", len(os.listdir(target)))

    print("Done.")

### Set up ImageDataGenerators for training and validation sets

In [ ]:
train_gen = trainGenerator(batch_size,'../../data/train','sat','map',
                        data_gen_args, save_to_dir = imgdatagen_dir, image_color_mode="rgba", target_size=target_size)

validation_gen = trainGenerator(batch_size,'../../data/validate','sat','map',
                        data_gen_args, save_to_dir = None, image_color_mode="rgba", target_size=target_size)

### Define model, compile, show summary, possibly load weights, define callbacks (including checkpoints)

In [ ]:
model = unet()
model.compile(optimizer=optimizer,
              loss=loss,
              loss_weights=loss_weights,
              metrics=metrics)
model.summary()
if (pretrained_model_fn):
    model.load_weights(pretrained_model_fn)
model_checkpoint = ModelCheckpoint(trained_model_fn, monitor='loss',verbose=1, save_best_only=True)

### Run training

In [ ]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=steps_per_epoch,
                              epochs=epochs,
                              callbacks=[model_checkpoint],
                              validation_data=validation_gen,
                              validation_steps=validation_steps
                             )

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
    plt.plot(history["acc"], label="acc")
    plt.plot(history["val_acc"], label="val_acc")
    plt.legend()
    plt.show()
    plt.close()
    
    plt.plot(history["loss"], label="loss")
    plt.plot(history["val_loss"], label="val_loss")
    plt.legend()
    plt.show()
    plt.close()

    plt.plot(history["auc_roc"], label="auc_roc")
    plt.plot(history["val_auc_roc"], label="val_auc_roc")
    plt.legend()
    plt.show()
    plt.close()

plot_history(history.history)